In [ ]:
# Input data and output folders
inUsers = "Users.txt"
inSongs = "Songs.txt"
inListenToSongs = "ListenToSongs.txt"

outputPathPart1 = "outPart1/"
outputPathPart2 = "outPart2/"

In [ ]:
# Write your code here

In [ ]:
# Part 1

In [ ]:
usersRDD = sc.textFile(inUsers)
songsRDD = sc.textFile(inSongs)
listenRDD = sc.textFile(inListenToSongs).cache()

In [ ]:
# Select young users
youngUsersRDD = usersRDD.filter(lambda line: int(line.split(",")[4])>1999)

In [ ]:
# Select the songs listened to in the last two years
def lastTwoYears(line):
    startTimestamp = line.split(",")[2]
    if (startTimestamp>="2018/09/14" and startTimestamp<="2020/09/13"):
        return True
    else:
        return False
   
    
listenLast2YearsRDD = listenRDD.filter(lastTwoYears)

In [ ]:
# Select the songs listened to by young users in the last two years

In [ ]:
# Map listenLast2YearsRDD to pairs (UID,SID)
def userSong(line):
    fields = line.split(",")
    sid = fields[0]
    uid = fields[1]
    
    return (uid,sid)
    
userSongTwoYearsRDD = listenLast2YearsRDD.map(userSong)

In [ ]:
# Map youngUsersRDD to pairs (UID,None)
def userNone(line):
    fields = line.split(",")
    uid = fields[0]
    
    return (uid,None)
    
youngUserPairRDD = youngUsersRDD.map(userNone)

In [ ]:
# Join the content of userSongTwoYearsRDD and youngUserPairRDD to identify the songs listened to by young users in the last two years
songsListenedYoungUsersRDD = userSongTwoYearsRDD.join(youngUserPairRDD)

In [ ]:
# Select the SIDs of the songs listened to by young users in the last two years
sidListenedYoungUsersRDD = songsListenedYoungUsersRDD.map(lambda pair: pair[1][0])

In [ ]:
# Extract all the possible SIDs from songsRDD
allSidsRDD = songsRDD.map(lambda line: line.split(",")[0])

In [ ]:
# Select the SIDs of the songs that do not occur in sidListenedYoungUsersRDD
selectedSIDsRDD = allSidsRDD.subtract(sidListenedYoungUsersRDD)

In [ ]:
selectedSIDsRDD.saveAsTextFile(outputPathPart1)

In [ ]:
# Part 2

In [ ]:
# Compute for each song its popularity in each year, i.e., the number of distinct users who listened to it in that year

In [ ]:
# Map the content of listenRDD to tuples (SID,Year,UID)
def tupleSidYearUser(line):
    fields = line.split(",")
    sid = fields[0]
    uid = fields[1]
    year = fields[2].split("/")[0]
    
    return (sid,year,uid)

sidYearUserRDD = listenRDD.map(tupleSidYearUser)

In [ ]:
# Remove duplicates. We are interested in the number of distinct users for each (song,year)
sidYearDistinctUserRDD =sidYearUserRDD.distinct()

In [ ]:
# Count the number of distinct user for each pair (sid,year)

In [ ]:
# Map sidYearDistinctUserRDD to pairs ( (sid,year), +1)
sidYearOneRDD = sidYearDistinctUserRDD.map(lambda record: ( (record[0],record[1]), +1))

In [ ]:
# ReduceByKey to count the number of distinct user for each pair (sid,year)
sidYearPopularityRDD = sidYearOneRDD.reduceByKey(lambda v1,v2: v1+v2)

In [ ]:
# Compute the highest popularity for each song and the first year associated with the highest popularity

In [ ]:
# Map sidYearPopularityRDD to pairs (SID, (annual popularity, year))
sidAnnualPopularityYearRDD = sidYearPopularityRDD.map(lambda pair: (pair[0][0], (pair[1], pair[0][1])))

In [ ]:
# Apply reduceByKey to compute the highest annual popularity for each SID.
# Given two input pair values (annualPopularityV1, year1) and (annualPopularityV2, year2) 
# I select the pair associated with the highest annualPopularity value. 
# If the annualPopularity is the same for both pair, I select the pair associated wioth the first year in the temporal order
def maxPopFirstYear(v1, v2):
    annualPopularityV1 = v1[0]
    year1 = v1[1]
    annualPopularityV2 = v2[0]
    year2 = v2[1]
    
    if (annualPopularityV1>annualPopularityV2):
        return v1
    else:
        if (annualPopularityV1<annualPopularityV2):
            return v2
        else:
            if (year1<year2):
                return v1
            else:
                return v2

sidFirstYearMaxPopRDD =  sidAnnualPopularityYearRDD.reduceByKey(maxPopFirstYear)

In [ ]:
# Extract only SID and year
sidYearRDD = sidFirstYearMaxPopRDD.map(lambda pair: (pair[0], pair[1][1]))

In [ ]:
sidYearRDD.saveAsTextFile(outputPathPart2)